### Imports

In [38]:

import os, sys, json, datetime, re, xlrd  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
import pandas as pd             # Provides data structures and data analysis tools
from openpyxl import Workbook
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import glob
import time
from tqdm import tqdm
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'

### gathering leg files

In [22]:
os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data')
legislator_files = glob.glob('**/*.xlsx') 

for i,file in enumerate(legislator_files):
    if '_legislators' not in str(file):
        print("deleting: " + str(legislator_files[i]))
        del legislator_files[i] 

deleting: archive\officials_download_quorum_10_4.xlsx
deleting: IL\Il_committee_data_from_quorum.xlsx
deleting: IN\IN_committee_data_from_quorum.xlsx


In [23]:
dfs = []
for i,file in enumerate(legislator_files):
    print('working on file:' + str(file))
    # file = legislator_files[0]
    # xls = pd.ExcelFile(file)
    sheets_dict = pd.read_excel(file, engine="openpyxl", sheet_name=None)
    sheet_names = list(sheets_dict.keys())
    for s in sheet_names:
        df = pd.read_excel(file, engine="openpyxl", sheet_name=s)
        dfs.append(df)
    df = pd.concat(dfs)

working on file:AL\AL_legislators.xlsx
working on file:CT\CT_legislators.xlsx
working on file:IL\IL_legislators.xlsx
working on file:IN\IN_legislators.xlsx
working on file:KS\KS_legislators.xlsx
working on file:MO\MO_legislators.xlsx
working on file:NC\NC_legislators.xlsx
working on file:ND\ND_legislators.xlsx
working on file:NM\NM_legislators.xlsx
working on file:OH\OH_legislators.xlsx
working on file:OK\OK_legislators.xlsx
working on file:VA\VA_legislators.xlsx
working on file:WV\WV_legislators.xlsx


In [24]:
all_legs = pd.concat(dfs)

In [25]:
os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\attendance data\exports')
# all_legs.to_csv('list_of_legislators_11_8_2024.csv')

### gathering attendance data

In [31]:
os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\attendance data')
events = glob.glob("*.xlsx")
state_list = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", 
    "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", 
    "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", 
    "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", 
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", 
    "New Hampshire", "New Jersey", "New Mexico", "New York", 
    "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", 
    "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", 
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
    "West Virginia", "Wisconsin", "Wyoming", "District of Columbia"
]
state_abbreviations = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "DC"
]

In [27]:
print("a statement")
state_abbreviations_reg = []
for abv in state_abbreviations:
    for_regex = f'^{abv}'
    state_abbreviations_reg.append(for_regex)

a statement


In [28]:
state_pat = re.compile("|".join(state_list))

In [29]:
state_abv_pat = re.compile("|".join(state_abbreviations_reg))
print(state_abv_pat)
state_ref = dict(zip(state_list, state_abbreviations))

re.compile('^AL|^AK|^AZ|^AR|^CA|^CO|^CT|^DE|^FL|^GA|^HI|^ID|^IL|^IN|^IA|^KS|^KY|^LA|^ME|^MD|^MA|^MI|^MN|^MS|^MO|^MT|^NE|^NV|^NH|^NJ|^NM|^NY|^NC|^ND|^OH|^OK|^OR|^PA|^RI|^SC|^SD|^TN|^TX|^UT|^VT|^VA|^WA|^WV|^WI|^WY)


### Filling in State info

In [39]:
dfs = []
vals_changed = 0
for event in events:
    df = pd.read_excel(event)
    # print('######################')
    
    # print(*df.columns)
    event_name = str(event).split('.')[0].strip().replace('_', ' ')
    df = df.iloc[:,:8]
    df.loc[:,'event name'] = event_name
    
    break_all = False
    # #print(df)
    # continue
    for i,state in enumerate(df['state']):
        #print(i)
        # print(type(state))
        # #print(str(state))
        # #print('----------------------------')
        #print(df.loc[i,['first_name', 'last_name', 'title', 'org']])
        #print(str(event_name))
        # #print(df.loc[i, 'event name'])
        # # continue
        # #print('----------------------------')
        if isinstance(state, float):
            if re.search(r'[Rr]epresentative|[Ss]enator|[Ll]egislator',str(df['title'].iloc[i])) or re.search(r'[Ss]enate|[Hh]ouse of ([Rr]epresentatives)?(Delegates)?|[Dd]istrict|[Ss]tate [Hh]ouse', str(df['org'].iloc[i])):
                # continue
                #print("^^^^^^^^^^^")
                #print("found a match")
                # #print(df.loc[i,['first_name', 'last_name']])
                
                testing_string = str(df['title'].iloc[i]) + " " + str(df['org'].iloc[i])
                # #print(testing_string)
                testing_string = testing_string.lstrip('nan').lstrip().strip()
                # #print(re.match(r'[Rr]epresentative|[Ss]enator|[Ll]egislator|[Ss]enate|[Hh]ouse of ([Rr]epresentatives)?(Delegates)?|[Dd]istrict|[Ss]tate [Hh]ouse',str(testing_string)))
                # continue
                # #print('###########')
                # #print(df.loc[i, list(df.columns[:5]) + [df.columns[-1]]])
                # #print('\n')
                state_match_uc = re.findall(state_pat, str(df['org'].iloc[i]))
                state_match = [x for x in state_match_uc if len(x) > 0]
                
                # First match test
                if len(state_match) == 0:
                    #print('no regular state match')
                    #print(state_match_uc)
                    state_abv_match_uc = re.findall(state_abv_pat, str(df['org'].iloc[i]))
                    state_abv_match = [x for x in state_abv_match_uc if len(x) > 0]
                    # Second match test
                    if len(state_abv_match) == 0:
                        #print('no state abbreviation match')
                        #print(state_abv_match_uc)
                        state_abv_event_match_uc = re.findall(state_abv_pat, str(df['event name'].iloc[i]))
                        state_abv_event_match = [x for x in state_abv_event_match_uc if len(x) > 0]
                        # Third match test
                        if len(state_abv_event_match) == 0:
                            #print('no state abv event match')
                            #print(state_abv_event_match_uc)
                            break
                        elif len(state_abv_event_match) > 1:
                            #print('more than one match?')
                            break_all = True
                            break
                        else:
                            #print("abv in event match")
                            state_val = str(state_abv_event_match[0])
                            # df.loc[i,'state'] = None
                            df.loc[i,'state'] = str(df.loc[i,'state'])
                            df.loc[i,'state'] = state_val
                            #print(state_val)
                            vals_changed += 1
                    elif len(state_abv_match) > 1:
                        #print('more than one match?')
                        #print(state_abv_match)
                        #print(df.loc[i, list(df.columns[:5]) + [df.columns[-1]]])
                        break_all = True
                        break
                    else:
                        #print("regular abreviation match")
                        
                        state_val = str(state_abv_match[0])
                        # df.loc[i,'state'] = None
                        df.loc[i,'state'] = str(df.loc[i,'state'])
                        df.loc[i,'state'] = state_val
                        #print(state_val)
                        vals_changed += 1

                    # #print('###########')
                    # #print(df.loc[i, list(df.columns[:5]) + [df.columns[-1]]])
                    # #print('\n')
                    # break
                elif len(state_match) > 1:
                    #print("more than one match?")
                    break_all = True
                    break
                else:
                    #print("normal state match")
                    state_val_dirty = str(state_match[0])
                    state_val = state_ref.get(state_val_dirty)
                    df.loc[i,'state'] = str(df.loc[i,'state'])
                    # df.loc[i,'state'] = None
                    df.loc[i,'state'] = state_val
                    #print(state_val)
                    vals_changed += 1
            else:
                # #print('#########################')
                # #print('NOT A REP OR SEN')
                # #print(df.loc[i,['first_name','last_name','title', 'org']])
                continue
                # #print(df.loc[i, list(df.columns[3:5]) + [df.columns[-1]]])
                # #print('\n')
    if break_all == True:
        break
    dfs.append(df)

In [40]:
event_data = pd.concat(dfs)
event_data.reset_index(inplace=True, drop = True)

In [41]:
for i,j in enumerate(event_data['state']):
    
    if isinstance(j, float):
        continue
    elif re.search(r'[A-Z]{2}', str(j)):
        continue
    else:
        val = state_ref.get(str(j))
        event_data.loc[i,'state'] = str(val)

%%

In [ ]:
os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\attendance data\exports')
# event_data.to_csv("event_data_export_11_7_2024.csv", index=False)

NameError: name 'event_data' is not defined

### refining data

In [42]:
title_pattern = r'[Rr]epresentative|[Ss]enator|[Ll]egislator'
org_pattern = r'[Ss]enate|[Hh]ouse of ([Rr]epresentatives)?(Delegates)?|(?<!School )(?:House District|District)|[Ss]tate [Hh]ouse'

In [87]:
filtered_df = event_data[event_data['title'].astype(str).apply(lambda x: bool(re.search(title_pattern, x))) |
                 event_data['org'].astype(str).apply(lambda x: bool(re.search(org_pattern, x)))]
no_districts = filtered_df[~(filtered_df['org'].str.contains(r'[Dd]istrict\s?\d{1,3}|[Dd](-|\s)?\d{2,3}', regex=True) | 
                filtered_df['title'].str.contains(r'[Dd]istrict\s?\d{1,3}|[Dd](-|\s)?\d{2,3}', regex=True))]
w_districts = filtered_df[(filtered_df['org'].str.contains(r'[Dd]istrict\s?\d{1,3}|[Dd](-|\s)?\d{2,3}', regex=True) | 
                filtered_df['title'].str.contains(r'[Dd]istrict\s?\d{1,3}|[Dd](-|\s)?\d{2,3}', regex=True))]

w_districts.reset_index(inplace=True, drop=True)
w_districts['chamber'] = ""

C:\Users\clutz\AppData\Local\Temp\ipykernel_36912\3899269669.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  no_districts = filtered_df[~(filtered_df['org'].str.contains(r'[Dd]istrict\s?\d{1,3}|[Dd](-|\s)?\d{2,3}', regex=True) |
C:\Users\clutz\AppData\Local\Temp\ipykernel_36912\3899269669.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df['title'].str.contains(r'[Dd]istrict\s?\d{1,3}|[Dd](-|\s)?\d{2,3}', regex=True))]
C:\Users\clutz\AppData\Local\Temp\ipykernel_36912\3899269669.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  w_districts = filtered_df[(filtered_df['org'].str.contains(r'[Dd]istrict\s?\d{1,3}|[Dd](-|\s)?\d{2,3}', regex=True) |
C:\Users\clutz\AppData\Local\Temp\ipykernel_36912\3899269669.py

In [ ]:
i = 0
for a,b in zip(w_districts.title, w_districts.org):
    # print('#######################')
    # print('***********')
    # print(a)
    # print('***********')
    # print(b)
    # continue
    has_a = False
    has_b = False
    if 'district' in str(a).lower() or re.search(r'[Dd]-?\s?\d{1,3}', str(a)):
        match_a = re.findall(r'[Dd]istrict\s?\d{1,3}|[Dd]-?\s?\d{1,3}', str(a))
        match_a = [x for x in match_a if len(x) > 0]
        if len(match_a) == 0:
            print('no results for title')
            print(a)
            
        else:
            has_a = True
            match = match_a[0]
            # print("a match: " + match)
            # print('################')
            # print(match_a)
        # print(str(dis))
    
    if 'district' in str(b).lower() or re.search(r'[Dd]-?\s?\d{1,3}', str(b)):
        match_b = re.findall(r'[Dd]istrict\s?\d{1,3}|[Dd]-?\s?\d{1,3}', str(b))
        match_b = [x for x in match_b if len(x) > 0]
        if len(match_b) == 0:
            print('no results for org')
            print(b)
        
        else:
            has_b = True
            match = match_b[0]
            # print("b match: " + match)
            # print('################')
            # print(match_b)
    # else:
    #     print('no results')
    #     print(a)
    #     print(b)


    if has_b == True or has_a == True:
        match_final = re.findall(r'\d+', str(match))
        print("final match: " + str(match_final[0]))
        print("putting it on row: " + str(i))
        w_districts.loc[i, 'district'] = str(match_final[0]).strip().lstrip('0')

    i +=1

### Fix for no districts
The chunk below brings in a manually edited file that incorporates districts from match where available
missing info mostly comes from states where we have not pulled legislator data

In [92]:
patch_file = r"C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\attendance data\archive\no_districts_attendance_patch.xlsx"
districts_patch = pd.read_excel(patch_file)

patched_df = pd.concat([w_districts,districts_patch])
patched_df.reset_index(inplace=True, drop=True)
i = 0
for a,b in zip(patched_df.title, patched_df.org):
    if re.search(r'[Hh]ouse|[Ss]enate', str(b)):
        if re.search(r'[Hh]ouse', str(b)):
            chamber = "House"
        elif re.search(r'[Ss]enate', str(b)):
            chamber = "Senate"
    elif re.search(r'[Rr]epresentative|[Ss]enator|[Dd]elegate', str(a)):
        if re.search(r'[Rr]epresentative|[Dd]elegate', str(a)):
            chamber = "House"
        elif re.search(r'[Ss]enator', str(a)):
            chamber = "Senate"

    try:
        patched_df.loc[i,'chamber'] = str(chamber)
        i += 1
    except:
        i += 1
        continue



In [ ]:

patched_df['helper'] = patched_df['state'].astype(str)+ "-"+ patched_df['chamber'].astype(str)+ "-"+patched_df['district'].astype(str)
patched_df.loc[patched_df['state'].isna() | (patched_df['state'] == "") | (patched_df['district'].isna()), 'helper'] = None

patch_minus_nan = patched_df[~(patched_df['helper'].isna())]
# print(patch_minus_nan.columns)
thi_states_df = patch_minus_nan.loc[:,['helper','first_name', 'last_name', 'honorific', 'title', 'org', 'district',
       'role', 'state', 'event name']]

grouped_df = thi_states_df.groupby('helper').agg({
    'first_name': 'first',
    'last_name': 'first',
    'honorific': 'first',
    'title': 'first',
    'org': 'first',
    'district': 'first',
    'state': 'first',
    'event name': lambda x: '|'.join(
        f"{sc} ({ac})" if not pd.isna(ac) else f"{sc}"
        for sc, ac in zip(thi_states_df.loc[x.index, 'event name'], thi_states_df.loc[x.index, 'role'])),

}).reset_index()
# grouped_df.reset_index()
grouped_df.rename(columns={'event name': 'events'}, inplace=True)





In [130]:
for i,j in enumerate(grouped_df['events']):
    
    score = 0
    speaker = False
    is_hfk = False
    dev_program = False
    in_state = False
    out_state = False
    is_slr = False
    dinner_or_lunch = False
    non_slr = False

    # print('#################')
    # print(*grouped_df.loc[i,['helper','first_name', 'last_name', 'events']], sep=" \ ")
    event_split = str(j).split('|')
    # print(len(event_split))
    # print('\n')
    speaker = False
    for event in event_split:
        
        match = re.findall(r'\(.+\)', str(event))
        match_refine = [x for x in match if len(x) != 0]
        if len(match_refine) == 0:
            continue
        
        for m in match_refine:
            if re.search('speaker|presenter', str(m).lower()):
                speaker = True
            elif 'HFK' in str(m):
                is_hfk = True
        
        if re.search(r'[Dd]inner|[Ll]unch', str(event)):
            dinner_or_lunch = True


        state = str(grouped_df.loc[i,'helper']).split('-')[0].strip()
        

        if 'ECLS' not in str(event) or "HFK" not in str(event):

            try:
                event_state = re.findall(state_abv_pat, str(event))[0].strip()
            except:
                
                print(str(event))
                print('no state match')
                break

            if event_state == state:
                print("states match")
                in_state = True

            else:
                out_state = False
            
        else:
            if 'ECLS' in str(event):
                out_of_state = True

        if 'HSPF' in str(event) or 'Elevate' in str(event):
            dev_program = True

        if re.search(r'SLR|HLR',str(event)):
            is_slr = True

        if re.search(r'\s[Mm]\d', str(event)):
            non_slr = True



    if speaker == True:
        if in_state == True:
            score += 10
        elif out_state == True:
            score += 15

    if is_slr == True:
        score += 15

    if is_hfk == True:
        score += 20

    if dev_program == True:
        score += 15

    if non_slr == True: 
        score += 10

    if dinner_or_lunch == True:
        score += 5
    
    
    print(score)
        

            

    continue

        # print("%%%%%%%%%%%%%%%")
        # print(*match_refine, sep=' - ')
        # print('%%%%%%%%%%%%%%%')

    continue

    speaker = False
    for event in event_split:
        if re.search(r'\(.+\)', str(event)):
            match = re.findall(r'\(.+\)', str(event))
            match = match[0]
            if 'speaker' in str(match).lower():
                speaker = True
            
    # if len(event_split) == 1

0
0
0
0
0
HKF C10 S1 (HKF)
no state match
0
0
0
0
0
0
0
0
0
0
0
0
0
ECLS 2024 (Team Lead, Presenter)
no state match
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
states match
25
0
states match
25
0
0
0
0
0
0
0
0
0
0
states match
25
0
0
0
0
0
0
0
0
0
states match
25
0
states match
ECLS 2024 (Presenter)
no state match
25
states match
25
states match
HKF C10 S1 (HKF)
no state match
25
0
HSPF C4 M2 (Speakers)
no state match
0
HSPF C4 M1 (Speakers)
no state match
0
0
states match
ElevateNC C4 M3 (Cohort Member)
no state match
15
states match
15
0
0
states match
ElevateNC C4 M3 (Cohort Member)
no state match
15
states match
15
states match
15
states match
15
states match
15
states match
15
states match
15
HSPF C4 M1 (Speakers)
no state match
0
states match
15
states match
15
states match
15
states match
15
states match
15
0
states match
15
states match
15
states match
10
ElevateNC C4 M3 (Cohort Member)
no state match
0
states match
15
HSPF C4 M3 (Speakers)
no state match
0
HSPF C4 M3 (Speakers)
no

### make lookup

the chunk below cuts down the dataframe to two columns for a look up value

In [ ]:

os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\attendance data\exports')
no_districts.to_csv("no_districts_export_11_7_2024.csv", index=False)

%%

In [ ]:
only_legs_event
if re.search(r'[Rr]epresentative|[Ss]enator|[Ll]egislator',str(df['title'].iloc[i])) or re.search(r'[Ss]enate|[Hh]ouse of ([Rr]epresentatives)?(Delegates)?|[Dd]istrict|[Ss]tate [Hh]ouse', str(df['org'].iloc[i])):

In [ ]:
only_legs_event = 